## COVID-19 Open Research Dataset Challenge - What do we know about vaccines and therapuetics?
The following questions were analysed specifically: 
- Effectiveness of drugs being developed and tried to treat COVID-19 patients.
  - Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.
- Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients.
- Exploration of use of best animal models and their predictive value for a human vaccine.
- Capabilities to discover a therapeutic (not vaccine) for the disease, and clinical effectiveness studies to discover therapeutics, to include antiviral agents.
- Efforts targeted at a universal coronavirus vaccine.
- Efforts to develop animal models and standardize challenge studies
- Assays to evaluate vaccine immune response and process development for vaccines, alongside suitable animal models (in conjunction with therapeutics)

## Our approach - Creating a timeline visualizing the progress of vaccines/cures on COVID-19 and other similar viral diseases.
Our goal is to create an intuitive visualization of the progress of research on vaccines and therapuetics regarding COVID-19. Not only is this useful for professional researchers in having a quick overview of the clinical trial stages of each investigated vaccine/therapeutic, but also for the public, to have a better understanding of the time frame for which to expect a cure or solution. We decided to create vizualizations of research progress of other virusses as well as COVID-19, to get a better picture of the timescale and ammount of research that goes into making a vaccine or therapeutics.

Several steps were taken to create the visualizations:
1. Load and preprocess the data:
    - lemmatize all texts and remove stopwords
2. Categorize papers based on keywords 
    - using either string pattern matching or word embeddings
    - relevant words were manually selected based on the research questions and indicativaty of clinical stage trial (e.g. mouse vs human test subject, words expressing certainty etc.)
    - categories are: virus, clinical stage, drug type
3. Extract keywords/summaries from selected papers
    - TODO: write how we do this @Simon, @Silvan
5. Visualize extracted papers, links and summaries
    - TODO: explain how (after we know how) @Levi @Gloria


### 0.a Imports

In [1]:
# TODO: write your imports here
import os
import json

import pandas as pd

from datetime import datetime

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

from nltk.stem import WordNetLemmatizer

import pickle as pk
import numpy as np

# path to data
data_dir = '../../src'  
keyword_dir = '../../keywords'

### 0.b Functions

In [2]:
# As kaggle only allows notebook submissions, all functions should be in the notebook. Just copy your functions and paste them here.
          
def load_data(data_dir):
    """Load data from dataset data directory."""
    sha = []
    full_text = []

    subdir = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,x))]

    print(f"[INFO] Loading data from {data_dir}...")
    # loop through folders with json files
    for folder in subdir:
#             path = os.path.join(data_dir,folder, folder)
        path = os.path.join(data_dir,folder, folder, 'pdf_json')
        # loop through json files and scrape data
        for file in os.listdir(path):
            file_path = os.path.join(path, file)

            # open file only if it is a file
            if os.path.isfile(file_path):
                with open(file_path) as f:
                    data_json = json.load(f)
                    sha.append(data_json['paper_id'])

                    # combine abstract texts / process
                    combined_str = ''
                    for text in data_json['body_text']:
                        combined_str += text['text'].lower()
                        
                    full_text.append(combined_str)

            else:
                print('[WARNING]', file_path, 'not a file. Check pointed path directory in load_data().')

    loaded_samples = len(sha)
    print(f"[INFO] Data loaded into dataset instance. {loaded_samples} samples added.")
    
    df = pd.DataFrame()
    df['sha'] = sha
    df['full_text'] = full_text
    
    return df

def clean_time(val):
    try:
        return datetime.strptime(val, '%Y-%m-%d')
    except:
        try:
            return datetime.strptime(val, '%Y %b %d')
        except:
            try:
                return datetime.strptime(val, '%Y %b')
            except:
                try:
                    return datetime.strptime(val, '%Y')
                except:
                    try:
                        return datetime.strptime('-'.join(val.split(' ')[:3]), '%Y-%b-%d')
                    except Exception as e:
                        return None
class Dataset:
    """COVID19 Papers Dataset
    
    Attributes:
        data_dir: string location where data files can be found.
        paper_ids: list containing str of unique pdf ids for each paper. 
            ie. ['sha1', 'sha2', 'sha3', ...]
        titles: list containing str titles of papers. 
            ie. ['title1', 'title2', 'title3', ...]
        abstracts: list containing str abstracts of each paper. 
            ie. ['abstract1', 'abstract2', 'abstract3', ...]
        n_paragraphs: list of integers specifying the amount of paragraphs in each paper. 
            ie. [n1, n2, n3, ...]
        contents: nested list containing contents of paper; contents of each paper stored in a list of strings containing paragraphs. 
            ie. [['paper1_p1', 'paper1_p2', ...], ['paper2_p1', 'paper2_p2', ...], ...]
        paper_dates: list containing str of datetime in most optimal format possible
            ie. ['date1', 'date2', 'date3', ...]
    
    Attributes are initially empty. To populate data, run class method of load_data().
    
    Usage:
        # declare directory where data is stored
        data_dir = '/kaggle/input/CORD-19-research-challenge'  
        data = Dataset(data_dir)
        data.load_data()
        
        # get attributes
        data.paper_ids
        data.titles
        ...
    """
    
    def __init__(self, data_dir:str):
        # init lists to store data
        self.data_dir = data_dir
        self.paper_ids = []
        self.titles = []
        self.abstracts = []
        self.n_paragraphs = []
        self.contents = []
        self.paper_dates = []
        
        print("[INFO] Empty Dataset object created.")
        
    @property
    def __len__(self):
        """Denotes the total number of samples."""
        return f"Dataset instance has {len(self.paper_ids)} samples"
    
    def load_data(self):
        """Load data from dataset data directory."""
        data_dir = str(self.data_dir)
        subdir = [x for x in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir,x))]
        
        initial_samples = len(self.paper_ids)
        
        print(f"[INFO] Loading data from {data_dir}...")
        # loop through folders with json files
        for folder in subdir:
            
            path = os.path.join(data_dir,folder, folder, 'pdf_json')
#             path = os.path.join(data_dir,folder, folder)
            
            try:

                # loop through json files and scrape data
                for file in os.listdir(path):
                    file_path = os.path.join(path, file)

                    # open file only if it is a file
                    if os.path.isfile(file_path):
                        with open(file_path) as f:
                            try:
                                data_json = json.load(f)
                            except:
                                # Broken file
                                self.paper_ids.append(None)
                                self.titles.append(None)
                                self.abstracts.append(None)
                                self.n_paragraphs.append(None)
                                self.contents.append([])
                            else:
                                self.paper_ids.append(data_json['paper_id'])
                                self.titles.append(data_json['metadata']['title'])

                                # combine abstract texts / process
                                combined_str = ''
                                for text in data_json['abstract']:
                                    combined_str += text['text'].lower()

                                self.abstracts.append(combined_str)

                                # take only text part for content
                                paragraphs = []
                                content = data_json['body_text']

                                for paragraph in content:
                                    paragraphs.append(paragraph['text'].lower())

                                self.n_paragraphs.append(len(content))
                                self.contents.append(paragraphs)
                    else:
                        print('[WARNING]', file_path, 'not a file. Check pointed path directory in load_data().')
            
            except FileNotFoundError:
                print('[WARNING]', path, 'does not exist, skipping search for files in this path...')
                
        end_samples = len(self.paper_ids)
        loaded_samples = end_samples - initial_samples
        print(f"[INFO] Data loaded into dataset instance. {loaded_samples} samples added. | Start amount = {initial_samples}; End amount = {end_samples}")
def get_published_dates(self):
    """ Populate publication date data. THIS SHOULD BE BROKEN AS DATACLASS IS REMOVED """
    print(f"[INFO] Filling paper publication date information by matching id with information from metadata.csv...")
    # read metadata
    metadata = pd.read_csv(os.path.join(self.data_dir, 'metadata.csv'))
    
    # crosscheck 'sha' column with paper_id
    for sha in self.paper_ids:
        date = metadata.loc[metadata['sha'] == sha, 'publish_time'].values
        
        if len(date):
            date = clean_time(date[0])
        else:
            date = None
        
        self.paper_dates.append(date)
        
    print(f"[INFO] Publication date information added.")

In [3]:
      
def tokenize_check(text):
    if isinstance(text, str):
        word_tokens = word_tokenize(text)
    elif isinstance(text, list):
        word_tokens = text
    else:
        raise TypeError
    return word_tokens
    

def remove_stopwords(text, remove_symbols=False):
    """ Tokenize and/or remove stopwords and/or unwanted symbols from string"""
    list_stopwords = set(stopwords.words('english'))
    # list of signs to be removed if parameter remove_symbols set to True
    list_symbols = ['.', ',', '(', ')', '[', ']']
    
    # check input type and tokenize if not already
    word_tokens = tokenize_check(text)

    # filter out stopwords
    text_without_stopwords = [w for w in word_tokens if not w in list_stopwords] 
    
    if remove_symbols is True:
        text_without_stopwords = [w for w in text_without_stopwords if not w in list_symbols]
    
    return text_without_stopwords

# from nltk.stem import WordNetLemmatizer 

def lemmatize(text):
    """ Tokenize and/or lemmatize string """
    lemmatizer = WordNetLemmatizer()
    
    # check input type and tokenize if not already
    word_tokens = tokenize_check(text)
    
    lemmatized_text = [lemmatizer.lemmatize(w) for w in word_tokens]
    
    return lemmatized_text

def find_keywords(text, df):
    """ Find relevant papers for the categories in df
    Returns a dictionary with the paper id's that match the categories
    It also stores the sentences where the matches have been found. This can be returned too if so the team decides """

    # Data cleaning:
    # Turn df into a dictionary with a list of key phrases
    # Lower all of them and remove null values
    dfd = {k: [x.lower() for x in v if not pd.isnull(x)] for k, v in df.to_dict('list').items()}
    
    matches = {}
    scores = {}
    
    # Remove redundant values (i.e., ['coronavirus', 'coronavirus disease'] can be left as ['coronavirus']; the element 'coronavirus disease' is useless)
    for k, v in dfd.items():
        # print(k)
        v = [x for x in v if not any([y in x for y in [z for z in v if z != x]])]
        dfd[k] = v

        # Find matches
        # Use the loop we're in where we've already cleaned the data to find the matches
        
        # if you use keyprhase, it handles phase i and phase ii the same way, it would count both..
        
        for sentence in sent_tokenize(text):
            for keyphrase in v:
                if keyphrase in sentence:
                    try:
                        already_a_match = sentence in matches[k]
                    except KeyError:
                        matches[k] = [sentence]
                    else:
                        if not already_a_match:
                            matches[k].append(sentence)
                            
        # score is scaled by the number of values to choose from
        if k in matches:
            scores[k] = len(matches)/len(v)

    # return the keys with the highest score. also return the sentences for this.
    if len(scores.keys()) > 0:
        max_score = list(scores.keys())[np.argmax(scores.values())]
        return max_score, matches[max_score]
    else:
        return 'nan','nan'

def kw_match_tables(df):
    """ Build table with boolean values indicating kw matches """

    keywords = {
        'virus': virus_keywords.columns.tolist(), 
        'stage': clinical_stage_keywords.columns.tolist(), 
        'drug': drug_keywords.columns.tolist(), 
    }

    headers = [item for sublist in [x.columns.tolist() for x in [virus_keywords, clinical_stage_keywords, drug_keywords]] for item in sublist]

    # Use titles instead of sha hashes because there's a lot of papers without sha that contain the keywords; yet all papers have a title
    table = pd.DataFrame(False, index=df.title, columns=headers)

    # Fill with True values
    for k, kws in keywords.items():
        for kw in kws:
            table.loc[df[df[k] == kw].title.tolist()] = True

    # Merge
    df = pd.merge(df, table, on='title')

    return df

def summarize(text):
    # TODO @Simon @Silvan: extract keywords
    return 'summary'

#def visualize_data(data,keywords,summaries):
#    #TODO @Levi @Kwan: visualize data

### 0.c Relevant strings

In [4]:
# keywords that define the virus the paper is about (likely in title)
virus_keywords = pd.read_csv(keyword_dir+'/virus_keywords.csv')

# keywords describing clinical phase
clinical_stage_keywords = pd.read_csv(keyword_dir+'/phase_keywords.csv')

# keywords describing treatment types
drug_keywords = pd.read_csv(keyword_dir+'/drug_keywords.csv')

### 1. Load and Preprocess the data

In [9]:
# try the preloaded dataframe to speed up the process
try:
    df = pk.load(open('df.pkl','rb'))
except:
    # create dataset object
    meta_data = pd.read_csv(data_dir+'/metadata.csv')
    meta_data['publish_time'] = meta_data['publish_time'].apply(clean_time)
    full_texts = load_data(data_dir)

    # merge full text and metadata, so the paper selection can be performed either on full text
    # or abstract, if the full text is not available.
    df = pd.merge(meta_data,full_texts,on='sha',how='outer')
    df['full_text'][df['full_text'].isna()] = df['abstract'][df['full_text'].isna()]

    # drop papers with no abstract and no full text
    df = df.dropna(subset=['abstract','full_text'])
    df = df[df['full_text'] != 'Unknown']
    pk.dump(df,open('df.pkl','wb'))

[INFO] Loading data from ../../src...
[INFO] Data loaded into dataset instance. 36236 samples added.


/home/liz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [10]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url,full_text
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4,Abstract The etiologic basis for the vast majo...
3,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4,Abstract Upper respiratory tract infections ar...
4,jhx90hh0,NaN,Elsevier,Monoclonal antibodies identify multiple epitop...,10.1016/0006-291x(85)91946-1,NaN,2409966.0,els-covid,Abstract Nine hybridoma cell lines secreting a...,1985-06-28,"Cherel, Isabelle; Grosclaude, Jeanne; Rouze, P...",Biochemical and Biophysical Research Communica...,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0006-291x(85)91946-1,Abstract Nine hybridoma cell lines secreting a...
15,iqswl5kh,NaN,Elsevier,Morphology and morphogenesis of a coronavirus ...,10.1016/0014-4800(76)90045-9,NaN,187445.0,els-covid,Abstract The morphology and morphogenesis of v...,1976-12-31,"Doughri, A.M.; Storz, J.; Hajer, I.; Fernando,...",Experimental and Molecular Pathology,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0014-4800(76)90045-9,Abstract The morphology and morphogenesis of v...
28,z65m48tn,NaN,Elsevier,Demonstration of viral antigen and immunoglobu...,10.1016/0021-9975(89)90122-9,NaN,2469703.0,els-covid,Abstract Haemagglutinating encephalomyelitis v...,1989-02-28,"Narita, M.; Kawamura, H.; Haritani, M.; Kobaya...",Journal of Comparative Pathology,NaN,NaN,False,False,custom_license,https://doi.org/10.1016/0021-9975(89)90122-9,Abstract Haemagglutinating encephalomyelitis v...


### 2. Define virus type, clinical stage and drug type

In [ ]:
try:
    df = pk.load(open('df_kw.pkl','rb'))
except:
    # function on full text --> think about applying on full text or on abstract
    df['virus'], df['virus_sentence'] = zip(*df['abstract'].apply(find_keywords,df=virus_keywords))
    df['stage'], df['stage_sentence'] = zip(*df['abstract'].apply(find_keywords,df=clinical_stage_keywords))
    df['drug'], df['drug_sentence'] = zip(*df['abstract'].apply(find_keywords,df=drug_keywords))
    
    # drop papers with nan values?
    pk.dump(df,open('df_kw.pkl','wb'))

In [ ]:
df

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,has_pmc_xml_parse,full_text_file,url,full_text,virus,virus_sentence,stage,stage_sentence,drug,drug_sentence
0,8q5ondtn,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,...,False,custom_license,https://doi.org/10.1016/0002-8703(72)90077-4,Abstract The etiologic basis for the vast majo...,nan,nan,preclinical,[Abstract The etiologic basis for the vast maj...,therapeutics,[Successful control of virus-induced congenita...
3,cjuzul89,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,...,False,custom_license,https://doi.org/10.1016/0002-9343(85)90361-4,Abstract Upper respiratory tract infections ar...,common cold,[Serotypes of the rhinoviruses account for 20 ...,preclinical,[Pneumonia remains an important cause of morbi...,therapeutics,"[Given the diversity of pathogenic agents, it ..."
4,jhx90hh0,NaN,Elsevier,Monoclonal antibodies identify multiple epitop...,10.1016/0006-291x(85)91946-1,NaN,2409966.0,els-covid,Abstract Nine hybridoma cell lines secreting a...,1985-06-28,...,False,custom_license,https://doi.org/10.1016/0006-291x(85)91946-1,Abstract Nine hybridoma cell lines secreting a...,nan,nan,nan,nan,nan,nan
15,iqswl5kh,NaN,Elsevier,Morphology and morphogenesis of a coronavirus ...,10.1016/0014-4800(76)90045-9,NaN,187445.0,els-covid,Abstract The morphology and morphogenesis of v...,1976-12-31,...,False,custom_license,https://doi.org/10.1016/0014-4800(76)90045-9,Abstract The morphology and morphogenesis of v...,nan,nan,nan,nan,nan,nan
28,z65m48tn,NaN,Elsevier,Demonstration of viral antigen and immunoglobu...,10.1016/0021-9975(89)90122-9,NaN,2469703.0,els-covid,Abstract Haemagglutinating encephalomyelitis v...,1989-02-28,...,False,custom_license,https://doi.org/10.1016/0021-9975(89)90122-9,Abstract Haemagglutinating encephalomyelitis v...,nan,nan,preclinical,[Abstract Haemagglutinating encephalomyelitis ...,nan,nan
29,8zilr4cy,NaN,Elsevier,Bovine herpesvirus-1-induced pharyngeal tonsil...,10.1016/0021-9975(92)90053-w,NaN,1602058.0,els-covid,Abstract The potential involvement of the phar...,1992-04-30,...,False,custom_license,https://doi.org/10.1016/0021-9975(92)90053-w,Abstract The potential involvement of the phar...,nan,nan,nan,nan,nan,nan
30,i60trxri,NaN,Elsevier,Pathogenicity and antigen detection of the Nou...,10.1016/0021-9975(92)90068-6,NaN,1313460.0,els-covid,Abstract We compared the pathogenicity and the...,1992-01-31,...,False,custom_license,https://doi.org/10.1016/0021-9975(92)90068-6,Abstract We compared the pathogenicity and the...,nan,nan,preclinical,[Abstract We compared the pathogenicity and th...,nan,nan
31,49nvudqj,NaN,Elsevier,Western and dot immunoblotting analysis of vir...,10.1016/0022-1759(84)90043-7,NaN,6208281.0,els-covid,Abstract Viral proteins were separated by sodi...,1984-10-12,...,False,custom_license,https://doi.org/10.1016/0022-1759(84)90043-7,Abstract Viral proteins were separated by sodi...,nan,nan,nan,nan,nan,nan
32,x5ygn82d,NaN,Elsevier,Plaque/focus immunoassay: a simple method for ...,10.1016/0022-1759(84)90301-6,NaN,6389707.0,els-covid,"Abstract A new, simple enzyme-linked immunosor...",1984-11-30,...,False,custom_license,https://doi.org/10.1016/0022-1759(84)90301-6,"Abstract A new, simple enzyme-linked immunosor...",nan,nan,preclinical,"[Abstract A new, simple enzyme-linked immunoso...",nan,nan
43,tcrw7lzd,NaN,Elsevier,Presence of infectious polyadenylated RNA in t...,10.1016/0042-6822(77)90498-6,NaN,193262.0,els-covid,Abstract Avian infectious bronchitis virus (IB...,1977-04-30,...,False,custom_license,https://doi.org/10.1016/0042-6822(77)90498-6,Abstract Avian infectious bronchitis virus (IB...,nan,nan,nan,nan,nan,nan


#### 2.2. Fill the table with boolean values for keyword matches

In [ ]:
# Update df with the tables with boolean values
df = kw_match_tables(df)
df

### 3. Summarize the texts

In [14]:
df['summary'] = df['full_text'].apply(summarize)

### 4. Visualize extracted papers, links and summaries

In [15]:
visualize_data(df)

NameError: name 'visualize_data' is not defined